In [1]:
from pyspark.sql import SparkSession
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.evaluation import MulticlassMetrics
import json

# 1. Spark Session initialisieren
spark = SparkSession.builder \
    .appName("Sentiment Analysis with JSON Data") \
    .getOrCreate()

sc = spark.sparkContext

# 2. JSON-Daten laden
# Beispiel: JSON-Datei mit einem Eintrag pro Zeile
path = "C:/Users/robar/Downloads/Video_Games1.json"

data_df = spark.read.json(path)

In [2]:
print(data_df.take(19))

[Row(asin='0439381673', image=None, overall=1.0, reviewText='I used to play this game years ago and loved it. I found this did not work on my computer even though it said it would work with Windows 7.', reviewTime='06 9, 2014', reviewerID='A21ROB4YDOZA5P', reviewerName='Mary M. Clark', style=None, summary='Did not like this', unixReviewTime=1402272000, verified=True, vote=None), Row(asin='0439381673', image=None, overall=3.0, reviewText='The game itself worked great but the story line videos would never play, the sound was fine but the picture would freeze and go black every time.', reviewTime='05 10, 2014', reviewerID='A3TNZ2Q5E7HTHD', reviewerName='Sarabatya', style=None, summary='Almost Perfect', unixReviewTime=1399680000, verified=True, vote=None), Row(asin='0439381673', image=None, overall=4.0, reviewText="I had to learn the hard way after ordering this for my MacBook Pro that this doesn't work unless you have MAC OS version 10.3 or less. I found that out after contact the Learnin

In [7]:
# 3. Relevante Felder auswählen
# Verwende nur "reviewText" und "overall" für die Sentiment-Analyse
selected_df = data_df.select("reviewText", "overall")

# 4. DataFrame filtern
filtered_df = selected_df.filter("reviewText IS NOT NULL AND overall IS NOT NULL")

# 5. DataFrame in RDD umwandeln
# Jede Zeile wird in ein Tupel (Bewertung, Text) umgewandelt
data_rdd = data_df.rdd.map(lambda row: (row["overall"], row["reviewText"]))

print(data_rdd.take(19))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 18) (Surface executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIter

In [13]:
#data = data.sample(False, 0.1, seed=42)
# 3. JSON-Parsing und relevante Felder extrahieren
def parse_json(line):
    obj = json.loads(line)
    review_text = obj.get("reviewText", "")  # Rezensionstext
    overall = obj.get("overall", 3.0)       # Bewertung (als Label)
    # Sentiment-Label (positiv: >=4, negativ: <4)
    label = 1.0 if overall >= 4.0 else 0.0
    return label, review_text

parsed_data = data.map(parse_json)

In [14]:

# 4. 80/20-Split
train_rdd, test_rdd = parsed_data.randomSplit([0.8, 0.2], seed=42)

# 5. Feature Engineering mit HashingTF
hashing_tf = HashingTF(10000)  # 10.000 Dimensionen

def featurize_data(label, text):
    words = text.split(" ")  # Text in Wörter zerlegen
    features = hashing_tf.transform(words)  # Feature-Vektoren generieren
    return LabeledPoint(label, features)

train_features = train_rdd.map(lambda x: featurize_data(x[0], x[1]))
test_features = test_rdd.map(lambda x: featurize_data(x[0], x[1]))

In [16]:
print(train_features.take(10))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 59) (Surface executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIt

In [15]:
 #6. Training des Modells
model = LogisticRegressionWithSGD.train(train_features, iterations=100)

# 7. Vorhersagen auf dem Testset
predictions_and_labels = test_features.map(lambda x: (model.predict(x.features), x.label))

# 8. Evaluierung
metrics = MulticlassMetrics(predictions_and_labels)
accuracy = metrics.accuracy
print(f"Test Accuracy: {accuracy:.2f}")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 58) (Surface executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIt

In [10]:


# 9. Beispiel-Vorhersagen anzeigen
print(predictions_and_labels.take(10))

# Spark Session beenden
spark.stop()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 44) (Surface executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIter